In [1]:
# !pip install -U airportsdata
# !pip install geopy

import requests
import json
import airportsdata
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim

# Pure data
airports = airportsdata.load()

# Processed Data
airports_df = pd.DataFrame(columns=["code", "lat", "long"])

for key in airports.keys():
  if ("international" in airports[key]["name"].lower()) or ("intl" in airports[key]["name"].lower()):
    if airports[key]["iata"] is not "":
      # print(airports[key])
      temp_df = pd.DataFrame([[airports[key]["iata"], airports[key]["lat"], airports[key]["lon"]]], columns=["code", "lat", "long"])
      airports_df = airports_df.append(temp_df, ignore_index=True)

<>:20: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:20: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/jc/wkxxw26x64n0pj9_zhhr878h0000gn/T/ipykernel_45703/2446196114.py:20: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if airports[key]["iata"] is not "":
/var/folders/jc/wkxxw26x64n0pj9_zhhr878h0000gn/T/ipykernel_45703/2446196114.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airports_df = airports_df.append(temp_df, ignore_index=True)
/var/folders/jc/wkxxw26x64n0pj9_zhhr878h0000gn/T/ipykernel_45703/2446196114.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airports_df = airports_df.append(temp_df, ignore_index=True)
/var/folders/jc/wkxxw26x64n0pj9_zhhr878h0000gn/T/ipykernel_45703/2446196114.py:23: FutureWarning: The frame.append method is depre

In [2]:
airports_df.head()

,code,lat,long
0,HIR,-9.428,160.054993
1,INU,-0.547458,166.919006
2,POM,-9.44338,147.220001
3,WWK,-3.58383,143.669006
4,KEF,63.985001,-22.6056


In [6]:
# starting_loc=input("Where are you leaving from? ")
# destination=input("Where would you like to visit? ")
# start_date=input("When will you be leaving? (YYYY-MM-DD) ")
# end_date=input("When will you be coming back? (YYYY-MM-DD) ")
# adult_count=int(input("How many adults will be coming on this trip? "))
# child_count=int(input("How many children will be coming on this trip? "))

starting_loc="Boulder"
destination="San Francisco"
start_date="2022-06-10"
end_date="2022-06-17"
adult_count=1
child_count=0

def getBestAirports(location, airports_df, nbest=3):
  geocoder = Nominatim(user_agent="GetLoc")
  getLocation = geocoder.geocode(location)

  lat = getLocation.latitude
  long = getLocation.longitude

  location = geocoder.reverse((lat, long))
  
  # print(location.raw["latitude"])

  city = location.raw["address"]["city"]
  if "state" in location.raw["address"].keys():
    state = location.raw["address"]["state"]
  elif "region" in location.raw["address"].keys():
    state = location.raw["address"]["region"]

  df = airports_df.copy()

  print(df.info())

  df["dist"] = np.sqrt((airports_df["lat"].astype('float') - lat)**2 + (airports_df["long"].astype('float') - long)**2)
  best_airports = df.nsmallest(nbest, columns=["dist"])
  best_airports.reset_index(inplace=True)

  return best_airports, city, state

home_best_airports, home_city, home_state = getBestAirports(starting_loc, airports_df)
dest_best_airports, dest_city, dest_state = getBestAirports(destination, airports_df)
# print(location.raw["address"]["country_code"].upper())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    841 non-null    object
 1   lat     841 non-null    object
 2   long    841 non-null    object
dtypes: object(3)
memory usage: 19.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    841 non-null    object
 1   lat     841 non-null    object
 2   long    841 non-null    object
dtypes: object(3)
memory usage: 19.8+ KB
None


In [ ]:
home_airport_code = home_best_airports.loc[0]["code"]
dest_airport_code = dest_best_airports.loc[0]["code"]

print(dest_best_airports)

   index code        lat        long      dist
0    293  SFO  37.618999 -122.375000  0.166208
1    261  OAK  37.721298 -122.221001  0.207113
2    294  SJC  37.362598 -121.929001  0.643739


In [ ]:
key = "774b07103bmsh7f4b47f00895eafp1ff206jsn58ad64733028"

url = "https://priceline-com-provider.p.rapidapi.com/v1/hotels/locations"

querystring = {"search_type":"HOTEL","name":f"{dest_city}, {dest_state}"}

headers = {
    'x-rapidapi-host': "priceline-com-provider.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

dictionary = json.loads(response.text)
city_id = dictionary[0]["cityID"]

print(city_id)


3000002244


In [ ]:
location_id = city_id
checkin = start_date
checkout = end_date
num_rooms = str(math.ceil(adult_count/2))

url = "https://priceline-com-provider.p.rapidapi.com/v1/hotels/search"

querystring = {"date_checkin":checkin,"location_id":location_id,"date_checkout":checkout,"sort_order":"PRICE","amenities_ids":"FINTRNT,FBRKFST","rooms_number":num_rooms,"star_rating_ids":"3.0,3.5,4.0,4.5,5.0"}

headers = {
    'x-rapidapi-host': "priceline-com-provider.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

hotels_for_city = json.loads(response.text)

# with open("drive/MyDrive/hotels.json", "w") as file:
# 	json.dump(hotels_for_city, file)

In [ ]:
# LIST OF ALL RELEVANT HOTELS
all_hotels = []
#

for num in range(len(hotels_for_city["hotels"])):
  hotel = hotels_for_city["hotels"][num]
  all_hotels.append(hotel)

In [ ]:
url = "https://priceline-com-provider.p.rapidapi.com/v2/flight/roundTrip"

# UNCOMMENT FOR A WORKING EXAMPLE
#home_airport_code = "DEN"
#dest_airport_code = "JFK"

print(dest_airport_code)

querystring = {"departure_date":f"{checkin},{checkout}","adults":f"{adult_count + child_count}","sid":"iSiX639","origin_airport_code":f"{home_airport_code},{dest_airport_code}","destination_airport_code":f"{dest_airport_code},{home_airport_code}"}

headers = {
    'x-rapidapi-host': "priceline-com-provider.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

response = json.loads(response.text)

SFO


In [ ]:
if "error" not in response["getAirFlightRoundTrip"].keys():
  min_cost = response["getAirFlightRoundTrip"]["results"]["air_search_rsp"]["total_trip_summary"]["minTotalFareWithTaxesAndFees"]
  flight = response["getAirFlightRoundTrip"]["results"]["air_search_rsp"]["total_trip_summary"]["carrier"][4]["airport"][0]["stops"][0]
  flight["airline"] = response["getAirFlightRoundTrip"]["results"]["air_search_rsp"]["total_trip_summary"]["carrier"][4]["code"]
  print(min_cost)

315.99


In [ ]:
flight

{'airline': 'UA',
 'lowestTotalFare': 325.19,
 'numberOfItineraries': 100,
 'stop': 0}

In [ ]:
response

{'getAirFlightRoundTrip': {'results': {'air_search_rsp': {'airline': [{'accepted_credit_cards': ['VI',
       'MC',
       'AX',
       'DS'],
      'airline_phone_numbers': [],
      'baggage_content_available': True,
      'baggage_fee_url': 'www.alaskaair.com/content/travel-info/baggage/overview',
      'check_in_url': 'www.alaskaair.com/',
      'code': 'AS',
      'frequent_flyer_program': 'Alaska Mileage Plan',
      'international_phone_number': '001-206-244-0751',
      'large_image': 'airLogo_ASlg.gif',
      'logo': 'https://assets-cdn.rezserver.com/media/img/air_logos2/AS.png',
      'name': 'Alaska Airlines',
      'phone_number': '1-800-252-7522',
      'seats_url': 'www.alaskaair.com/content/travel-info/baggage/overview',
      'small_image': 'airLogo_AS.png',
      'website_url': 'www.alaskaair.com/'},
     {'airline_phone_numbers': [],
      'baggage_content_available': False,
      'code': 'W(',
      'logo': 'https://assets-cdn.rezserver.com/media/img/air_logos2/W(.pn

In [ ]:
airlines = response["getAirFlightRoundTrip"]["results"]["air_search_rsp"]["total_trip_summary"]["carrier"]

# LIST OF ALL RELEVANT FLIGHT OPTIONS
all_flights = []
#

code_dict = {}

codes = response["getAirFlightRoundTrip"]["results"]["air_search_rsp"]["airline"]

for info in codes:
  code_dict[info["code"]] = info["name"]

for i in range(len(airlines)):
  airline = airlines[i]["airport"][0]["stops"]
  for flight in airline:
    flight["airline"] = code_dict[response["getAirFlightRoundTrip"]["results"]["air_search_rsp"]["total_trip_summary"]["carrier"][i]["code"]]
    all_flights.append(flight)

In [ ]:
url = "https://priceline-com-provider.p.rapidapi.com/v2/cars/resultsRequest"

querystring = {"pickup_date":checkin,"dropoff_date":checkout,"dropoff_time":"10:00","pickup_time":"10:00","pickup_airport_code":dest_airport_code,"currency":"USD"}

headers = {
    'x-rapidapi-host': "priceline-com-provider.p.rapidapi.com",
    'x-rapidapi-key': key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

rentals = json.loads(response.text)

In [ ]:
rental_dict = rentals["getCarResultsRequest"]["results"]["results_list"]

# LIST OF ALL RELEVANT CAR RENTAL OPTIONS
all_rentals = []
#

print(len(rental_dict))

for car_num in range(len(rental_dict)):  
  rental = rental_dict[f"result_{car_num}"]
  all_rentals.append(rental)

all_rentals

153


[{'car': {'air_conditioning': 'true',
   'allow_cancellation': True,
   'bags': 2,
   'campaign_rate': 'false',
   'description': 'Full-Size Car',
   'discount': 0,
   'doors': '4',
   'example': 'Toyota Camry or similar',
   'free_cancellation': True,
   'imageURL': 'https://s1.pclncdn.com/rc-static/vehicles/partner/av/size134x72/zdusfcar.jpg',
   'images': {'SIZE134X72': 'https://s1.pclncdn.com/rc-static/vehicles/partner/av/size134x72/zdusfcar.jpg',
    'SIZE268X144': 'https://s1.pclncdn.com/rc-static/vehicles/partner/av/size268x144/zdusfcar.jpg',
    'SIZE335X180': 'https://s1.pclncdn.com/rc-static/vehicles/partner/av/size335x180/zdusfcar.jpg',
    'SIZE536X288': 'https://s1.pclncdn.com/rc-static/vehicles/partner/av/size536x288/zdusfcar.jpg',
    'SIZE67X36': 'https://s1.pclncdn.com/rc-static/vehicles/partner/av/size67x36/zdusfcar.jpg'},
   'inclusions': [],
   'mileage': True,
   'partner_discounts': None,
   'passengers': 5,
   'pay_at_booking': False,
   'transmission': 'Automati

In [ ]:
example_recs_for_user = {19568237450: [{"hotel": hotel, "flight": flight, "rental": rental}, 
               {"hotel": hotel, "flight": flight, "rental": rental}, 
               {"hotel": hotel, "flight": flight, "rental": rental}, 
               {"hotel": hotel, "flight": flight, "rental": rental}]}

In [ ]:
import pickle

f = open("example_recommendations.pkl","wb")
pickle.dump(dict,f)
f.close()

In [ ]:
example_recs_for_user

{19568237450: [{'flight': {'airline': 'United Airlines',
    'lowestTotalFare': 635.18,
    'numberOfItineraries': 26,
    'stop': 2},
   'hotel': {'allInclusiveRateProperty': False,
    'brand': 'Residence Inn',
    'displayRank': 9975.0,
    'hotelFeatures': {'hotelAmenityCodes': ['FBRKFST',
      'FINTRNT',
      'FINTRPUB',
      'HEALTHSVCS',
      'SPOOL',
      'KITCHEN',
      'RESTRNT',
      'FITSPA',
      'PETALLOW',
      'HANDFAC',
      'NSMKFAC']},
    'hotelId': '41723',
    'location': {'address': {'addressLine1': '2000 Winward Way',
      'cityName': 'San Mateo',
      'countryName': 'United States',
      'isoCountryCode': 'US',
      'provinceCode': 'CA',
      'zip': '94404'},
     'cityId': 3000002259,
     'latitude': 37.5564,
     'longitude': -122.2843,
     'neighborhoodId': '910050141',
     'neighborhoodName': 'San Mateo',
     'timeZone': 'America/Los_Angeles',
     'zoneId': '50141'},
    'media': {'source': 'fastly',
     'url': 'https://mobileimg.pclncd

In [ ]:
print(len(all_flights), len(all_rentals), len(all_hotels))

9 153 26


In [ ]:
import datetime
start_d = datetime.datetime.strptime(start_date, "%Y-%m-%d")
end_d = datetime.datetime.strptime(end_date, "%Y-%m-%d")



In [ ]:
(end_d - start_d).days

7

In [ ]:
# BASIC RANKING ALGORITHM THAT ORDERS 
# OPTIONS BY TOTAL PRICE (EXCLUDING FOOD/FUEL)

from more_itertools import sort_together

min_cost = 100000000000
user_id = 1389724923659

costs = []
options = []

for flight_num in range(len(all_flights)):
  for rental_num in range(len(all_rentals)):
    for hotel_num in range(len(all_hotels)):
      flight_cost = float(all_flights[flight_num]["lowestTotalFare"])
      hotel_cost = float(all_hotels[hotel_num]["ratesSummary"]["minPrice"]) * (end_d - start_d).days
      rental_cost = float(all_rentals[0]["price_details"]["base"]["total_price"])

      cost = flight_cost + hotel_cost + rental_cost
      option = {"hotel": all_hotels[hotel_num], "flight": all_flights[flight_num], 
                "rental": all_rentals[rental_num], "total_cost": cost}

      costs.append(cost)
      options.append(option)


sorted_options = sort_together([costs, options])[1]

print(min(costs))
print(len(options))

options_dict = {user_id: options}

#print(round(min_cost, 2))

1819.32
35802


In [ ]:

print(options_dict[user_id][30000])

{'hotel': {'hotelId': '42085', 'name': 'Sonesta ES Suites San Francisco - Airport', 'brand': 'Sonesta', 'starRating': 3.0, 'location': {'address': {'addressLine1': '1350 Huntington Ave', 'cityName': 'San Bruno', 'provinceCode': 'CA', 'countryName': 'United States', 'zip': '94066', 'isoCountryCode': 'US'}, 'longitude': -122.419753, 'latitude': 37.641317, 'timeZone': 'America/Los_Angeles', 'neighborhoodId': '910051354', 'neighborhoodName': 'SFO North - San Bruno', 'cityId': 3000002238, 'zoneId': '51354'}, 'thumbnailUrl': 'https://mobileimg.priceline.com/htlimg/42/42085/thumbnail-150-square.jpg', 'hotelFeatures': {'hotelAmenityCodes': ['FBRKFST', 'FINTRNT', 'FINTRPUB', 'FPRKING', 'HEALTHSVCS', 'SPOOL', 'AIRSHUTTL', 'KITCHEN', 'RESTRNT', 'FITSPA', 'PETALLOW', 'HANDFAC', 'NSMKFAC']}, 'overallGuestRating': 8.7, 'totalReviewCount': 295, 'proximity': 1.0603495539107402, 'ratesSummary': {'minPrice': '190.43', 'minCurrencyCode': 'USD', 'minStrikePrice': '273.86', 'pclnId': 'D55D5157F84C0CA360BA5